
# 🥈 Etapa **PLATA** · Limpieza y Normalización (Web Logs)

Este notebook implementa la fase **PLATA** del pipeline **BRONCE → PLATA → ORO**.

**Objetivo:** tomar los eventos crudos (ya ingeridos en BRONCE) y:
- **Tipar** correctamente las columnas clave.
- **Normalizar** `user_id`, `path`, `referrer`, `device` (consistencia, minúsculas, quitar basura).
- **Enviar a cuarentena** los registros inválidos por campo (con *trazabilidad*).
- **Filtrar por día objetivo** (`day`) y enviar a cuarentena los fuera de día.
- **Ordenar y deduplicar** por `["user_id", "ts", "path"]`.
- Añadir columna `date` (string y derivada de `ts`).

Al final, devolvemos un `DataFrame` **PLATA** con registros **válidos** y dejamos los **inválidos** en ficheros *Parquet* (carpeta de cuarentena).



## 📦 Importaciones
Incluimos `pandas` y definimos utilidades de **normalización** y **escritura Parquet** dentro del propio notebook para que sea autosuficiente.


In [ ]:

import os
import re
from pathlib import Path
from typing import Any
import pandas as pd

# Ajusta si necesitas una carpeta base distinta
BASE_DIR = Path('/mnt/data')
BASE_DIR.mkdir(parents=True, exist_ok=True)
print("BASE_DIR:", BASE_DIR)



## 🧰 Utilidades de archivos (`utils/files.py` embebido)
Estas funciones crean carpetas, aseguran rutas y escriben **Parquet**. En un proyecto real vivirían en `utils/files.py`.


In [ ]:

from pathlib import Path
import os

def make_path_dirs(path_dirs: str) -> Path:
    # En este notebook resolvemos relativo a BASE_DIR para que sea reproducible
    return (BASE_DIR / path_dirs)

def ensure_dir(path_dirs: str, file_name: str) -> str:
    out_dir = make_path_dirs(path_dirs)
    os.makedirs(out_dir, exist_ok=True)
    return str(out_dir / file_name)

def write_parquet(df: pd.DataFrame, path_dir: str, file_name: str):
    out_path = ensure_dir(path_dir, file_name)
    df.to_parquet(out_path, index=False, engine="pyarrow")
    return out_path



## 🧽 Utilidades de normalización (`utils/normalizes.py` embebido)
Implementamos las mismas funciones que compartiste para normalizar cadenas, paths, referrers y device.


In [ ]:

from typing import Any

def normalize_string(x: Any) -> str | None:
    if not isinstance(x, str):
        return None
    if x == "":
        return None
    s = x.strip().lower()
    return s or None

def normalize_string_path(x: Any) -> str | None:
    s = normalize_string(x)
    if s is None:
        return None
    s = s.split("?", 1)[0]          # quita querystring
    s = re.sub(r"/{2,}", "/", s)    # colapsa // en /
    if s.startswith(("http://", "https://", "file://")):
        return None                  # URLs completas no se aceptan como path/referrer
    return s or None

def normalize_path(x: Any) -> str | None:
    s = normalize_string_path(x)
    if s and not s.startswith("/"):
        s = "/" + s
    return s or None

def normalize_referrer(x: Any) -> str | None:
    s = normalize_string_path(x)
    if s in {"", "(not set)"}:
        return None
    known = {"direct", "google", "facebook"}
    if s not in known and s and not s.startswith("/"):
        s = "/" + s
    return s or None

def normalize_device(x: Any) -> str | None:
    if not isinstance(x, str):
        return None
    s = x.strip().lower()
    return s if s in {"mobile", "desktop", "tablet"} else None



## 🥈 Función principal: `to_silver(df, day, quarantine_dir)`
- **Entrada:** `df` (DataFrame crudo/bronze), `day` (ej. `"2025-11-08"` en UTC), `quarantine_dir` (carpeta para errores).
- **Salida:** `DataFrame` validado/normalizado del **día objetivo**, sin duplicados.
- **Efecto colateral:** escribe ficheros Parquet de **cuarentena** por cada tipo de error encontrado.


In [ ]:

def to_silver(df: pd.DataFrame, day: str, quarantine_dir: str) -> pd.DataFrame:
    """Limpieza y normalización; los registros inválidos van a cuarentena."""
    out = df.copy()

    # Tipado/normalización
    out["ts"] = pd.to_datetime(out["ts"], errors="coerce", utc=True)
    out["user_id"] = out["user_id"].apply(normalize_string).astype("string")
    out["path"] = out["path"].apply(normalize_path).astype("string")
    out["referrer"] = out["referrer"].apply(normalize_referrer).astype("string")
    out["device"] = out["device"].apply(normalize_device).astype("string")

    # Enviar a cuarentena registros inválidos campo a campo
    for key in out.columns:
        if key in {"ts", "user_id", "path", "referrer", "device"}:
            mask_errors = out[key].isna()
            invalid = out.loc[mask_errors].copy()
            out.dropna(subset=[key], inplace=True)
            if not invalid.empty:
                invalid["_error"] = key if key != "ts" else "ts"
                path_written = write_parquet(invalid, f"{quarantine_dir}/{day}", f"error_{key}.parquet")
                print(f"[WARN] {len(invalid)} filas inválidas enviadas a → {path_written}")

    # Filtrar por día objetivo (UTC)
    day0 = pd.Timestamp(day, tz="UTC")
    mask_day = (out["ts"] >= day0) & (out["ts"] < day0 + pd.Timedelta(days=1))
    valid_day = out.loc[mask_day].copy()
    invalid_day = out.loc[~mask_day].copy()
    if not invalid_day.empty:
        invalid_day["_error"] = "outside_day"
        path_written = write_parquet(invalid_day, f"{quarantine_dir}/{day}", "error_out_ts.parquet")
        print(f"[WARN] {len(invalid_day)} filas inválidas enviadas a → {path_written}")

    # Orden y deduplicación
    valid_day = (
        valid_day.sort_values(["user_id", "ts", "path"])
        .drop_duplicates(subset=["user_id", "ts", "path"], keep="first")
    )

    # Columna de fecha (string)
    valid_day["date"] = valid_day["ts"].dt.date.astype("string")
    return valid_day



### 🔍 Paso a paso (resumen)

1. **Parseo y tipado** de `ts` (UTC) y normalización de `user_id`, `path`, `referrer`, `device`.
2. **Cuarentena por columna:** se detectan `NaN` tras normalizar; esas filas se escriben en Parquet con `_error` específico.
3. **Filtro por día (`day`)** en UTC: lo que no cae en `[day, day+1)` se manda a `error_out_ts.parquet`.
4. **Orden y dedupe** por `user_id, ts, path`.
5. **`date`**: columna derivada para facilitar agregaciones posteriores.



## 🧪 Demostración mínima
Creamos un pequeño DataFrame con casos buenos y malos para ver cómo funciona `to_silver`.


In [ ]:

# Datos de ejemplo
raw = pd.DataFrame([
    {"ts": "2025-11-08T10:00:00Z", "user_id": " U1 ", "path": " / ", "referrer": "Google",   "device": "Mobile"},
    {"ts": "2025-11-08T10:05:00Z", "user_id": "u1",  "path": "/productos", "referrer": "direct",   "device": "desktop"},
    {"ts": "2025-11-08T10:05:00Z", "user_id": "u1",  "path": "/productos", "referrer": "direct",   "device": "desktop"},  # duplicado exacto
    {"ts": "2025-11-08T10:40:00Z", "user_id": "u2",  "path": "contacto",   "referrer": "(not set)","device": "tablet"},
    {"ts": "2025-11-09T01:00:00Z", "user_id": "u3",  "path": "/carrito",   "referrer": "facebook", "device": "MOBILE"},   # fuera de día
    {"ts": None,                   "user_id": "u4",  "path": "/x",         "referrer": "??",       "device": "unknown"},  # ts inválido, device inválido
    {"ts": "2025-11-08T12:00:00Z", "user_id": "",    "path": "/y?utm=1",   "referrer": "site.com", "device": "desktop"},  # user_id vacío
])

day = "2025-11-08"
quarantine_dir = "quarantine_plata"

silver = to_silver(raw, day, quarantine_dir)
print("✅ Filas válidas (PLATA):", len(silver))
silver



### 📂 Salidas de cuarentena
Los ficheros Parquet con errores se escriben bajo `quarantine_plata/<day>/...` relativo a `BASE_DIR`.

Puedes inspeccionarlos con:


In [ ]:

list((BASE_DIR / quarantine_dir / day).glob("*.parquet"))
